<a href="https://colab.research.google.com/github/sonmh79/VRPPD-with-robots/blob/main/Delivery_Robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import time

In [2]:
e1 = pd.read_excel('/content/drive/MyDrive/Indoor_robot_parameter.xlsx',sheet_name='demand')
d_data = pd.read_excel('/content/drive/MyDrive/Indoor_robot_parameter.xlsx',sheet_name='OD_travel_time',index_col=0)
e3 = pd.read_excel('/content/drive/MyDrive/Indoor_robot_parameter.xlsx',sheet_name='optimal_solution')

In [3]:
#Pickup and Delivery
import random
e1.index =range(1,13)
e1
for i in range(len(e1.index)):
  e1.iloc[i,2] = random.randint(1,2)
e1

,Customer ID,Pickup demand,Delivery demand
1,1,0,1
2,2,2,2
3,3,1,2
4,4,0,1
5,5,0,1
6,6,1,2
7,7,1,1
8,8,0,1
9,9,2,2
10,10,0,2


In [4]:
df = e1.drop("Customer ID",axis=1)
df

,Pickup demand,Delivery demand
1,0,1
2,2,2
3,1,2
4,0,1
5,0,1
6,1,2
7,1,1
8,0,1
9,2,2
10,0,2


In [5]:
#Travel Time
d_data

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00,65.69,85.69,219.38,244.38,253.58,294.72,334.72,283.92,262.78,293.12,358.12,302.32
1,65.69,0.00,64.55,158.24,183.24,192.44,233.58,273.58,222.78,201.64,231.98,296.98,241.18
2,85.69,64.55,0.00,218.24,243.24,252.44,293.58,333.58,282.78,261.64,291.98,356.98,301.18
3,219.38,158.24,218.24,0.00,59.55,137.10,178.24,218.24,167.44,146.30,176.64,241.64,185.84
4,244.38,183.24,243.24,59.55,0.00,162.10,203.24,243.24,192.44,171.30,201.64,266.64,210.84
5,253.58,192.44,252.44,137.10,162.10,0.00,125.69,84.55,183.24,162.10,192.44,257.44,201.64
6,294.72,233.58,293.58,178.24,203.24,125.69,0.00,125.69,224.38,203.24,233.58,298.58,242.78
7,334.72,273.58,333.58,218.24,243.24,84.55,125.69,0.00,264.38,243.24,273.58,338.58,282.78
8,283.92,222.78,282.78,167.44,192.44,183.24,224.38,264.38,0.00,65.69,204.38,269.38,213.58
9,262.78,201.64,261.64,146.30,171.30,162.10,203.24,243.24,65.69,0.00,183.24,248.24,192.44


In [6]:
#Optimal Trip
e3

,# of robots,# of trips,Trip composition
0,1,1.0,{0 -> 2 -> 1 -> 0}
1,NaN,2.0,{0 -> 4 -> 10 -> 11 -> 5 -> 7-> 6-> 0}
2,2,1.0,{0 -> 12 -> 9 -> 8 -> 3 -> 0}
3,Solution value,NaN,2523.45 sec (about 39% more)


In [7]:
#@title  { form-width: "1px" }
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


class VRP:
  def __init__(self,dataset,d_data = d_data,capacity=3,cnum=100,mutation_prob=0.2,ev_times=100):
    self.n = len(dataset)
    self.c = capacity
    self.data = dataset
    self.d_data = d_data
    self.cnum = cnum
    self.cnt = 0
    self.mutation_prob = mutation_prob
    self.ev_times = ev_times
    self.dist = 0
    self.min_route = []
    self.info = []

  def _chromo(self,t=1):

    """ 1. Make Random Chromos """

    res = []
    for _ in range(t):
     res.append(random.sample(range(1,self.n),self.n-1)) 
    return res


  def show_trip(self,route):

    """ Print the Trip of Min Route """

    stack = 0
    new_trips = [0]
    for i in range(len(route)):
      cur_demand = self.data.loc[route[i]]["Delivery demand"]
      if stack + cur_demand <= self.c:
        stack += cur_demand
        new_trips.append(route[i])
      else:
        stack= cur_demand
        new_trips = new_trips+[0]
        new_trips.append(route[i])
    new_trips += [0]
    print(new_trips)


  def _calc_route(self,route):

    """ 2-1-1 Calculate Distance Considering D Demands """
    stack = 0
    new_trips = [0]
    for i in range(len(route)):
      cur_demand = self.data.loc[route[i]]["Delivery demand"]
      if stack + cur_demand <= self.c:
        stack += cur_demand
        new_trips.append(route[i])
      else:
        stack= cur_demand
        new_trips = new_trips+[0]
        new_trips.append(route[i])
    new_trips += [0]

    res = 0
    for i in range(len(new_trips)-1):
      res += self.d_data.loc[new_trips[i]][new_trips[i+1]]
    return res

  def _getdist(self, chromos):

    """ 2-1 Return Distance of Each Chromo """

    dist_list = []

    for chromo in chromos:
      dist_list.append(self._calc_route(chromo))

    fit = list(zip(dist_list,chromos))
    fit.sort(key = lambda x:x[0])
    return fit

  def _rand(self,x,y):
    return int(random.uniform(x,y))

  def _crossover(self,p1,p2,hard_mode=False):

    """ 2-2-2. Crossover and Mutation """

    swap_point = self._rand(1,len(p1))
    c1,c2 = [],[]

    i = 0
    while i < swap_point:
      c1.append(p1[i])
      c2.append(p2[i])
      i+=1

    for e in p2:
      if e not in c1:
        c1.append(e)

    for e in p1:
      if e not in c2:
        c2.append(e)
    
    #mutation
    if random.uniform(0,1) <= self.mutation_prob and len(c1)>2 and not hard_mode:
      e1,e2 = random.sample(range(1,len(c1)),2)
      #target will select mutation child
      if e1 > e2:
        target = e1
      else:
        target = e2
      
      if target%2 == 0:
        c1[e1],c1[e2] = c1[e2],c1[e1]
      else:  
        c2[e1],c2[e2] = c2[e2],c2[e1]

    if hard_mode:
      e1,e2 = random.sample(range(0,len(c1)-1),2)
      c1[e1],c1[e1+1] = c1[e1+1],c1[e1]
      c2[e2],c2[e2+1] = c2[e2+1],c2[e2]

    next_gen = self._getdist([c1,c2,p1,p2])
    #print("next_gen: ",next_gen)
    c1,c2 = next_gen[0][1],next_gen[1][1]
    #print("c1,c2: ",c1,c2)
    return [c1,c2]

  
  def _select_parent(self,parents):

    """ 2-2-1. Select Parents with Roulette Algorithm  """

    fitness = [1/route[0] for route in parents] 
    sum = 0

    for f in fitness:
      sum += f
    p_index = set()

    while len(p_index)<2:
      fit = 0
      target = random.uniform(0,sum)
      for i in range(len(fitness)):
        fit += fitness[i]
        if fit > target:
          p_index.add(i)
          break  
    return list(p_index)

  
  def _make_child(self,dist_routes):

    """ 2-2. Compare Parents with Childs and Return Best Routes  """

    self.cnt += 1
    parents = dist_routes.copy()
    routes = [route[1] for route in parents]
    child = []

    # select top 100 chromos
    while parents and len(child)<self.cnum:
      selected_parents = self._select_parent(parents)
      p1,p2 = routes.pop(selected_parents[0]),routes.pop(selected_parents[1]-1)
      parents.pop(selected_parents[0]),parents.pop(selected_parents[1]-1)
      c1,c2 = self._crossover(p1,p2)
      child += c1,c2

    return child

  def evolution(self,chromo):

    """ 2. Make Childs """
    
    dist_routes = self._getdist(chromo) #(dist,routes)
    child = self._make_child(dist_routes)
    return child

  def ga(self): 

    """ 0. Start Genetic Algorithm !! """

    chromos = self._chromo(100)    
    gen = self.evolution(chromos)
    self.info = self._getdist(gen)
    print("first genenration minimun distance: ",(self.info[0][0],self.info[0][1]))
    for i in range(self.ev_times):
      gen = self.evolution(gen)
      self.info = self._getdist(gen)
      t = self.cnt
      if t % 10 == 0:
        print("{}/{}th evolution min distance: ".format(t,self.ev_times),(self.info[0][0],self.info[0][1]))
      
      # to avoid local minimum
      if t == 30:
        self.mutation_prob = t/100
        print("==========Mutation prob upgrade!==========: ",self.mutation_prob)
      elif t == 50:
        self.mutation_prob = t/100
        print("==========Mutation prob upgrade!==========: ",self.mutation_prob)
      elif t == 70:
        self.mutation_prob = t/100
        print("==========Mutation prob upgrade!==========: ",self.mutation_prob)
      
    self.dist,self.min_route = self.info[0][0],self.info[0][1]   
    print("----------------------------------------Finish----------------------------------------")
    print("Total travel time / Min route: ",self.dist," / ",self.min_route)
    self.show_trip(self.info[0][1])
#if __name__ == "__main__":
  


In [8]:
import time
v = VRP(df,d_data,capacity=3,ev_times=10)
s= time.time()
v.ga()
e = time.time()
print("Total time: ",e-s,"(s)")

first genenration minimun distance:  (3522.1400000000003, [11, 5, 7, 8, 9, 2, 3, 4, 6, 10, 1])
10/10th evolution min distance:  (3376.99, [6, 7, 8, 9, 2, 3, 4, 10, 11, 5, 1])
----------------------------------------Finish----------------------------------------
Total travel time / Min route:  3376.99  /  [6, 7, 8, 9, 2, 3, 4, 5, 1, 10, 11]
[0, 6, 7, 0, 8, 9, 0, 2, 0, 3, 4, 0, 5, 1, 0, 10, 11, 0]
Total time:  13.230468273162842 (s)
